In [1]:
import sys
from otis import otis
from otis import kinematics

from reachy_sdk import ReachySDK


reachy = ReachySDK(host='localhost')

In [2]:
%matplotlib

from matplotlib import pyplot as plt

import numpy as np

# This can be installed via pip
from svgpathtools import svg2paths, wsvg

Using matplotlib backend: Qt5Agg


In [3]:
def svg_to_strokes(filename):
    paths, attrs = svg2paths(filename, convert_rectangles_to_paths=False)

    def pixel2mm(pix, DPI=300):
        return pix * 25.4 / DPI

    strokes = []

    for p in paths:
        pts = np.asarray([[np.real(p.point(x)), np.imag(p.point(x))] 
                          for x in np.linspace(0, 1, 30 * len(p))])
        X = pixel2mm(pts[:, 0])
        Y = pixel2mm(pts[:, 1])

        strokes.append(np.array((X, Y)).T)
        
    return strokes

In [4]:
svg_example_path = 'flower.svg'

strokes = svg_to_strokes(svg_example_path)

# plt.figure()
for s in strokes:
    X, Y = s.T
#     plt.plot(X, Y)
# plt.gca().set_aspect('equal', adjustable='box')

In [5]:
otis_hand = [reachy.r_arm.r_otis_motor_a, reachy.r_arm.r_otis_motor_b, reachy.r_arm.r_otis_motor_lift]

In [6]:
for motor in otis_hand:
    motor.compliant = False

In [7]:
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import minimum_jerk

In [8]:
otis.lift(reachy)

In [9]:
otis.drop(reachy)

In [10]:
five_bars_mechanism_params = {
    'Ax': 40,
    'Ay': 45,
    'Bx': 90,
    'By': 35,
    'R_A': 30,
    'R_B': 30,
    'L_A': 60,
    'L_B': 50,
    'd': 15,
    'theta_G': 80,
}

In [11]:
fbm = kinematics.FiveBarsMechanism(params=five_bars_mechanism_params)

In [12]:
import time

goal, real = [], []

for s in strokes:
    otis.lift(reachy)

    x, y = s[0]
    theta_a, theta_b = fbm.inverse(x, y)

    goto(
        goal_positions={
            reachy.r_arm.r_otis_motor_a: theta_a, 
            reachy.r_arm.r_otis_motor_b: theta_b
        }, 
        duration=0.5,
    )

    otis.drop(reachy)
    
    for x, y in s:
        a, b, = fbm.inverse(x, y)
        reachy.r_arm.r_otis_motor_a.goal_position = a
        reachy.r_arm.r_otis_motor_b.goal_position = b
        time.sleep(0.025)
    
        goal.append((a, b))
        real.append((reachy.r_arm.r_otis_motor_a.present_position, reachy.r_arm.r_otis_motor_b.present_position))

otis.lift(reachy)

goal = np.array(goal)
real = np.array(real)

In [15]:
# plt.figure()

# GX, GY = fbm.forward(goal[:, 0], goal[:, 1])
# RX, RY = fbm.forward(real[:, 0], real[:, 1])

# plt.plot(GX, GY, '--')
# plt.plot(RX, RY)

# plt.gca().set_aspect('equal', adjustable='box')